In [1]:
import numpy as np
import glob

In [2]:
def get_SNR(file):
    with open(file, "rt")as f:
        a = f.readlines()
    lst = []
    for x in a:
        if "Value:" in x:
            lst.append(float(x.split("Value: ")[1]))
        #print(x.split("bla"))
    lst = lst[len(lst)//2 : ]
    return (np.array(lst).mean(), np.array(lst).std())

In [3]:
get_SNR("rcv_0.txt")

(3.468054047619048, 0.4091092542810278)

In [4]:
glob.glob("*.txt")

['rcv_4.txt',
 'rcv_2.txt',
 'rcv_6.txt',
 'rcv_1.txt',
 'rcv_5.txt',
 'rcv_3.txt',
 'rcv_0.txt']

In [5]:
dc = {}

In [6]:
for file in glob.glob("*.txt"):
    mean, std = get_SNR(file)
    dc[file] = get_SNR(file)
    #print(f"{file} (SNR Mean, SNR Std): {mean}, {std}")

In [7]:
sorted(dc.items(), key = lambda x: x[1][0])

[('rcv_2.txt', (1.5815411190476192, 0.4853298958173268)),
 ('rcv_0.txt', (3.468054047619048, 0.4091092542810278)),
 ('rcv_3.txt', (4.559230476190477, 0.3337094797442857)),
 ('rcv_4.txt', (5.434059523809524, 0.3332578299794776)),
 ('rcv_1.txt', (5.4509823809523805, 0.3027723468203346)),
 ('rcv_5.txt', (5.679271904761904, 0.36516625846995127)),
 ('rcv_6.txt', (7.716105476190478, 0.2748177624501492))]

In [8]:
for x in sorted(dc.items(), key = lambda x: x[1][0]):
    print(f"{x[0]}:   SNR Mean, SNR Std: {x[1]}")

rcv_2.txt:   SNR Mean, SNR Std: (1.5815411190476192, 0.4853298958173268)
rcv_0.txt:   SNR Mean, SNR Std: (3.468054047619048, 0.4091092542810278)
rcv_3.txt:   SNR Mean, SNR Std: (4.559230476190477, 0.3337094797442857)
rcv_4.txt:   SNR Mean, SNR Std: (5.434059523809524, 0.3332578299794776)
rcv_1.txt:   SNR Mean, SNR Std: (5.4509823809523805, 0.3027723468203346)
rcv_5.txt:   SNR Mean, SNR Std: (5.679271904761904, 0.36516625846995127)
rcv_6.txt:   SNR Mean, SNR Std: (7.716105476190478, 0.2748177624501492)
